### Import Packages

In [3]:
import pandas as pd # data manipulation
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np #  mathematical support for large, multi-dimensional arrays and matrices
import os

In [4]:
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import collections

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-1srpdc8_ because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import re

### Set Directory

In [6]:
os.getcwd()

'/scratch/interns202010/jmakins/jupyter-dir'

In [7]:
try:
    os.chdir('/group/interns202010/jmakins/Data')
except:
    pass

In [8]:
os.listdir()

['world_soccer_database.sqlite',
 'events',
 'referees.json',
 'players.json',
 'competitions.json',
 'teams.json',
 'coaches.json',
 'matches',
 '.DS_Store']

In [9]:
Teams = pd.read_json('teams.json')
Competitions = pd.read_json('competitions.json')
Players = pd.read_json('players.json')
Coaches = pd.read_json('coaches.json') # Managers

In [10]:
#Competitions['CompId'] = Competitions['wyId']

In [11]:
#Competitions.drop(['wyId', 'CountryId'], inplace = True, axis = 1)

In [12]:
try:
    os.chdir('/group/interns202010/jmakins/Data/matches')
except:
    pass
England = pd.read_json('matches_England.json')
France = pd.read_json('matches_France.json')
Italy = pd.read_json('matches_Italy.json')
Spain = pd.read_json('matches_Spain.json')
Germany = pd.read_json('matches_Germany.json')
World_Cup = pd.read_json('matches_World_Cup.json')
Euro_Champs = pd.read_json('matches_European_Championship.json')

In [13]:
# Adding country labels to match observations
England['Country'] = 'England'
France['Country'] = 'France'
Italy['Country'] = 'Italy'
Spain['Country'] = 'Spain'
Germany['Country'] = "Germany"
World_Cup['Country'] = 'World Cup'
Euro_Champs['Country'] = 'European Champs'

In [166]:
try:
    os.chdir('/group/interns202010/jmakins/Data/events')
except:
    pass

In [167]:
os.listdir()

['events_Spain.json',
 'events_Germany.json',
 'events_France.json',
 'events_European_Championship.json',
 'events_England.json',
 'events_World_Cup.json',
 '.DS_Store',
 'events_Italy.json']

In [168]:
test = pd.read_json('events_England.json')

In [293]:
#Matches.loc[((Matches['homeTeamId']==1644) & (Matches['awayTeamId']==1659)) | ((Matches['homeTeamId']==1659) & (Matches['awayTeamId']==1644))]


In [294]:
#Events_England[Events_England['matchId']==2500037]

In [295]:
#Events_England[(Events_England['subEventDescription'].str.match('goal ')) & (Events_England['Result']=="Failure") & (Events_England['subEventName']=="Shot")]



In [465]:
#Events_England.loc[(Events_England['subEventDescription'].str.match("goal ")) & (Events_England['Result']=="Success")].subEventName.unique()



In [16]:
Events_France = pd.read_json('events_France.json')
Events_Spain = pd.read_json('events_Spain.json')
Events_Germany = pd.read_json('events_Germany.json')
#Events_EuroChamps = pd.read_json('events_European_Championship.json')
#Events_World_Cup = pd.read_json('events_World_Cup.json')
Events_Italy = pd.read_json('events_Italy.json')
Events_England = pd.read_json('events_England.json')

In [17]:
# Example of an events table
Events_England[Events_England['matchId'] == 2499719]

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
...,...,...,...,...,...,...,...,...,...,...,...,...
1763,8,Launch,[{'id': 1802}],8480,"[{'y': 53, 'x': 7}, {'y': 49, 'x': 46}]",2499719,Pass,1631,2H,2992.491575,84,177961040
1764,8,Head pass,[{'id': 1801}],49876,"[{'y': 51, 'x': 54}, {'y': 58, 'x': 73}]",2499719,Pass,1609,2H,2994.900590,82,177961035
1765,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1216}, {'id'...",7870,"[{'y': 58, 'x': 73}, {'y': 0, 'x': 0}]",2499719,Shot,1609,2H,2997.086392,100,177961036
1766,5,Ball out of the field,[],0,"[{'y': 48, 'x': 3}, {'y': 100, 'x': 100}]",2499719,Interruption,1631,2H,2998.963701,50,177961041


### Data Cleaning

In [18]:
Matches = pd.concat([England, France, Italy, Spain, Germany], ignore_index=True)

In [19]:
Matches.drop(["groupName", 'duration'], inplace=True, axis =1) # removing groupName variable from World Cups

In [20]:
Players.drop(["passportArea"], inplace = True, axis = 1)

In [21]:
# Example of Players dataframe
Players[0:3]

,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null


In [22]:
Players['Position'] = Players.role.apply(pd.Series)['code3'].values

In [23]:
Players['birthCountry'] = Players.birthArea.apply(pd.Series)['name'].values

In [24]:
Players.drop(["birthArea", 'role'], inplace = True, axis = 1)

### Manipulate Match Data

In [25]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match

In [26]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for x in teams:
    homeTeam.append(x[0])
    awayTeam.append(x[1])

In [27]:
# add columns for additional match statistics
Matches["homeManagerId"] = None
for name in ["Score", "ScoreHT", "ScoreET", "ScoreP"]:
    colname = "home" + name
    colname2 = "away" + name
    Matches[colname] = None
    Matches[colname2] = None

In [28]:
# parse Home and Away teams as features from nest dictionary
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for game, team in enumerate(teams):
    if Matches.teamsData[game:game+1][game][team[0]]['side'] == 'home':
        homeTeam.append(team[0])
        awayTeam.append(team[1])
    else:
        awayTeam.append(team[0])
        homeTeam.append(team[1])

In [29]:
def get_team_stats(df, team): # function to parse team match stats from dictionary embedded in column teamsData from the Matches dataframe
    
    Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId = [],[],[],[],[],[],[],[]
    
    for game in range(0, len(df)):

        Bench.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['bench'])['playerId']))
        Starters.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['lineup'])['playerId']))
        Manager.append(df.teamsData[game][team[game]]['coachId'])
        Score.append(df.teamsData[game][team[game]]['score'])
        ScoreHT.append(df.teamsData[game][team[game]]['scoreHT'])
        ScoreET.append(df.teamsData[game][team[game]]['scoreET'])
        ScoreP.append(df.teamsData[game][team[game]]['scoreP'])
        teamId.append(df.teamsData[game][team[game]]['teamId'])
    
    return Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId

homeBench, homeStarters, homeManager, homeScore, homeScoreHT, homeScoreET, homeScoreP, homeTeamId = get_team_stats(Matches, homeTeam)
awayBench, awayStarters, awayManager, awayScore, awayScoreHT, awayScoreET, awayScoreP, awayTeamId = get_team_stats(Matches, awayTeam)


In [30]:
# Automating Bench Line Up Column Generation

homeColNames = []
for num in range(1,14): # add columns for home bench playerId
     homeColNames.append("homeBenchPlayer" + str(num))
        
awayColNames = []
for num in range(1,14): # add columns for away bench playerId
    awayColNames.append("awayBenchPlayer" + str(num))
    


In [31]:
# Inspecting bench counts as during errror handling reveals some matches had bench sizes of between 8 and 13 for certain fixtures
import collections
collections.Counter(list(map(lambda x: len(x), homeBench)))

Counter({7: 1425, 6: 18, 5: 1, 8: 5, 12: 292, 10: 53, 11: 123, 9: 23, 13: 1})

In [32]:
# Adding Bench Line Ups to DataFrame

homeBench2 = pd.DataFrame(homeBench,
     columns=homeColNames)
awayBench2 = pd.DataFrame(awayBench,
     columns=awayColNames)


In [33]:
# Automating Starting Line Up Column Generation

homeColNames =[]
for num in range(1,12): # add columns for starting playerId
     homeColNames.append("homePlayer" + str(num))
        
awayColNames = []
for num in range(1,12): # add columns for starting playerId
    awayColNames.append("awayPlayer" + str(num))


In [34]:
# Joining starting line ups, benches to matches dataframe

homeStarters2 = pd.DataFrame(homeStarters,
     columns=homeColNames)
awayStarters2 = pd.DataFrame(homeStarters,
     columns=awayColNames)

Matches = Matches.join([homeStarters2, awayStarters2, homeBench2, awayBench2])


In [35]:
# add individual columns of match data
Matches['homeManager'] = homeManager
Matches['homeScore'] = homeScore
Matches['homeScoreHT'] = homeScoreHT
Matches['homeScoreET'] = homeScoreET
Matches['homeScoreP'] = homeScoreP
Matches['homeTeamId'] = homeTeamId
Matches['awayManager'] = awayManager
Matches['awayScore'] = awayScore
Matches['awayScoreHT'] = awayScoreHT
Matches['awayScoreET'] = awayScoreET
Matches['awayScoreP'] = awayScoreP
Matches['awayTeamId'] = awayTeamId

In [36]:
Matches.drop(["teamsData"], inplace = True, axis = 1)

In [37]:
# Classify Match Result as W, L or D for teams
conditions = [
    (Matches['winner'] == 0),
    (Matches['winner'] == Matches['awayTeamId']),
    (Matches['winner'] == Matches['homeTeamId'])
    ]

# create a list of the values we want to assign for each condition
values = [0, -1, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
Matches['Result'] = np.select(conditions, values)

###### Relating table to access Referee Data

In [ ]:
# os.chdir('/group/interns202010/jmakins/Data')

In [ ]:

# sql_conn = sqlite3.connect('world_soccer_database.sqlite') # soccer database containing over 100,000 match results which is updated monthly

# Match_TeamStats = pd.read_sql_query("SELECT * FROM football_data", sql_conn)

# # parsing betting odds variables from analysis
# drop_columns = ["B365H","B365D","B365A","BSH","BSD","BSA","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA",
# "PSH", "PH","PSD", "PD","PSA", "PA","SOH","SOD","SOA","SBH","SBD","SBA","SJH","SJD","SJA","SYH","SYD","SYA","VCH",
# "VCD","VCA","WHH","WHD","WHA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","MaxH","MaxD","MaxA",
# "AvgH","AvgD","AvgA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","GB>2.5","GB<2.5","B365>2.5","B365<2.5",
# "P>2.5","P<2.5","Max>2.5","Max<2.5","Avg>2.5","Avg<2.5","BbAH","BbAHh","AHh","BbMxAHH","BbAvAHH","BbMxAHA",
# "BbAvAHA","GBAHH","GBAHA","GBAH","LBAHH","LBAHA","LBAH","B365AHH","B365AHA","B365AH","PAHH", "AvgC<2.5",
# "PAHA","MaxAHH","MaxAHA","AvgAHH","AvgAHA","WHCH","WHCD","WHCA", "VCCH", "VCCA", "PSCH", "PSCD", "PCAHH", "PCAHA", "PC>2.5", "PC<2.5", "MaxCH", "MaxCAHH", "MaxCD",
# "MaxCAHA", "MaxCA", "MaxC>2.5", "MaxC<2.5", "IWCH", "IWCD", "IWCA", "AvgC>2.5","AvgCA","AvgCAHA","AvgCAHH",
# "AvgCD","AvgCH","B365C<2.5","B365C>2.5","B365CA","B365CAHA","B365CAHH","B365CD","B365CH","BWCA", "BWCD", "BWCH", "VCCD", "PSCA"] 

# Match_TeamStats.drop(drop_columns, inplace=True, axis =1) # removing betting odds from dataframe





In [ ]:
# Match_TeamStats = Match_TeamStats[Match_TeamStats['Datetime'].str.contains('2017|2018')]

In [39]:
Matches["date"] = pd.to_datetime(Matches['dateutc']).dt.date # create a date column for Matches dataframe

In [40]:
Matches["time"] = pd.to_datetime(Matches['dateutc']).dt.time # create a time column for Matches dataframe

In [41]:
# Dimensionality Reduction
cols2Drop = ['status', 'roundId', 'gameweek', 'dateutc', 'label', 'referees', 'homeManagerId', 'seasonId']
benches2Drop = []
for benchP in range(8,14):
    benches2Drop.append("awayBenchPlayer" + str(benchP))
    benches2Drop.append("homeBenchPlayer" + str(benchP))
    
cols2Drop = cols2Drop + benches2Drop

In [42]:
Matches.drop(cols2Drop, inplace=True, axis =1)

In [43]:
Matches.drop(['winner', 'date', 'time'], inplace = True, axis =1)

In [101]:
Matches.rename(columns={'wyId':'matchId'}, inplace=True)

In [44]:
# Match_TeamStats['date'] = pd.to_datetime(Match_TeamStats['Datetime']).dt.date

In [45]:
# # Get Referee and Home/Away Team Information
# Match_TeamStats.merge(Matches, how = "inner", indicator = False)

In [46]:
# Matches = Match_TeamStats.merge(Matches, how = "inner", indicator = False)

In [47]:
# Matches.drop(['gameweek', 'seasonId', 'dateutc', 'label', 'Season', 'status', 'roundId'], inplace= True, axis =1)

In [48]:
# Matches.drop(['Datetime', 'Div','FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'ABP', 'AC',
#        'AF', 'AFKC', 'AHCh', 'AHW', 'AO', 'AR', 'AS', 'AST', 'AT', 'AY',
#        'Attendance', 'Date', 'HBP', 'HC', 'HF', 'HFKC', 'HHW', 'HO', 'HR',
#        'HS', 'HST', 'HT', 'HY', 'Time', 'referees'], inplace= True, axis =1)

### Events Manipulation for Match/Player Stats and then combine with existing Match(test) dataframe from aboves

In [49]:
# Group by and events summarization

In [50]:
Events_England['subEventName'].unique()

array(['Simple pass', 'High pass', 'Head pass', 'Air duel',
       'Ground loose ball duel', 'Smart pass', 'Launch',
       'Ground defending duel', 'Ground attacking duel', 'Foul',
       'Free Kick', 'Cross', 'Shot', 'Reflexes', 'Touch', 'Clearance',
       'Ball out of the field', 'Throw in', 'Goal kick', 'Corner',
       'Goalkeeper leaving line', 'Hand pass', 'Acceleration',
       'Save attempt', '', 'Free kick cross', 'Free kick shot',
       'Hand foul', 'Violent Foul', 'Protest', 'Whistle',
       'Late card foul', 'Out of game foul', 'Penalty', 'Time lost foul',
       'Simulation'], dtype=object)

In [51]:
# Decroos Github
tags = [
    (101, "goal"),
    (102, "own_goal"),
    (301, "assist"),
    (302, "key_pass"),
    (1901, "counter_attack"),
    (401, "left_foot"),
    (402, "right_foot"),
    (403, "head/body"),
    (1101, "direct"),
    (1102, "indirect"),
    (2001, "dangerous_ball_lost"),
    (2101, "blocked"),
    (801, "high"),
    (802, "low"),
    (1401, "interception"),
    (1501, "clearance"),
    (201, "opportunity"),
    (1301, "feint"),
    (1302, "missed_ball"),
    (501, "free_space_right"),
    (502, "free_space_left"),
    (503, "take_on_left"),
    (504, "take_on_right"),
    (1601, "sliding_tackle"),
    (601, "anticipated"),
    (602, "anticipation"),
    (1701, "red_card"),
    (1702, "yellow_card"),
    (1703, "second_yellow_card"),
    (1201, "position_goal_low_center"),
    (1202, "position_goal_low_right"),
    (1203, "position_goal_mid_center"),
    (1204, "position_goal_mid_left"),
    (1205, "position_goal_low_left"),
    (1206, "position_goal_mid_right"),
    (1207, "position_goal_high_center"),
    (1208, "position_goal_high_left"),
    (1209, "position_goal_high_right"),
    (1210, "position_out_low_right"),
    (1211, "position_out_mid_left"),
    (1212, "position_out_low_left"),
    (1213, "position_out_mid_right"),
    (1214, "position_out_high_center"),
    (1215, "position_out_high_left"),
    (1216, "position_out_high_right"),
    (1217, "position_post_low_right"),
    (1218, "position_post_mid_left"),
    (1219, "position_post_low_left"),
    (1220, "position_post_mid_right"),
    (1221, "position_post_high_center"),
    (1222, "position_post_high_left"),
    (1223, "position_post_high_right"),
    (901, "through"),
    (1001, "fairplay"),
    (701, "lost"),
    (702, "neutral"),
    (703, "won"),
    (1801, "accurate"),
    (1802, "not_accurate"),
]

In [52]:
tags = dict(tags)

In [53]:
# ## I have kept this cell encase we need to impute integers for ML algorithms as oppose to strings
# event_tag_ids = []
# for ids in  list(trial[0:10]['tags']):
#     event_tag_ids.append(list(map(lambda y: y['id'], ids)))
# trial['event_tag_ids'] = event_tag_ids

In [54]:
trial = Events_England[Events_England['matchId'] == 2499719]

In [55]:
# Store event type tags ids in new column
def event_tags(event_df):
    event_tags = []
    for ids in  list(event_df['tags']):
        event_tags.append(list(map(lambda y: tags[y['id']], ids)))

    event_df['event_tags'] = event_tags
    return event_df

In [57]:
# Clean the result of the action
def Result(event_df):
    result = []
    for tag in event_df["event_tags"]:
        if "accurate" in tag:
            result.append("Success")
        elif "not_accurate" in tag:
            result.append("Failure")
        else:
            result.append("")
    event_df["Result"] = result 
    return event_df

In [59]:
# Separating description from success/failure of an action (event)
def event_description(event_df):
    subEventDescr = []
    for tag in event_df["event_tags"]:
        descr = " ".join(tag[:-1])
        if descr != "":
            subEventDescr.append(descr)
        else:
            subEventDescr.append("generic play")
    event_df["subEventDescription"] = subEventDescr
    event_df = event_df[(event_df["event_tags"].str.len() != 0) & (event_df.subEventName != 'Ball out of the field')  & (event_df.subEventName != 'Goal kick') & (event_df.subEventName != 'Touch')   ]
    event_df = event_df[event_df["subEventName"] != "Throw in"] # remove throw-ins as a relevant feature among successful teams for simpler analysis, unless its Rory Delap!
    return event_df

In [56]:
#Events_England = event_tags(Events_England)

In [58]:
#Events_England = Result(Events_England)

In [60]:
#Events_England = event_description(Events_England)

In [61]:
#### I prefer the segmented format of the cell below for locations but we will keep just encase***
# start, end = [], []
# for pos1, pos2 in trial['positions']:
#     start.append(pos1)
#     end.append(pos2)

# trial["StartLoc"] = start
# trial["EndLoc"] = end

In [62]:
collections.Counter(list(map(lambda x: len(x), Events_England['positions'])))

Counter({2: 548318, 1: 118})

In [285]:
# for pos in Events_England[~(Events_England.positions.str.len() == 2)]['positions']:
#     print(pos[0])

In [64]:
test = []
for pos in Events_England["positions"][0:10]:
    test.append(pos[0]['y'])

In [65]:
xStart,xEnd,yStart,yEnd = [], [], [], []
for pos in Events_England["positions"][0:10]:
    if len(pos) == 2:
        xStart.append(pos[0]['x'])
        xEnd.append(pos[1]['x'])
        yStart.append(pos[0]['y'])
        yEnd.append(pos[1]['y'])
    else:
        xStart.append(pos[0]['x'])
        xEnd.append(pos[0]['x'])
        yStart.append(pos[0]['y'])
        yEnd.append(pos[0]['y'])

In [74]:

def Locations(event_df):

    xStart,xEnd,yStart,yEnd = [], [], [], []
    for pos in event_df["positions"]:
        if len(pos) == 2:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[1]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[1]['y'])
        else:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[0]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[0]['y'])


    event_df['xStart'], event_df['xEnd'], event_df['yStart'], event_df['yEnd'] = xStart, xEnd, yStart, yEnd
    return event_df

In [67]:
Events_England = Locations(Events_England)

In [68]:
Events_England.drop(['eventId', 'positions', 'id', "event_tags", "tags", "eventId", "eventName"], inplace = True, axis = 1)

In [69]:
Events_England = Events_England[Events_England.subEventName != ""]

In [73]:
# function to transform an event data frame
def event_df_transform(event_df):
    event_df = event_tags(event_df)
    event_df = Result(event_df)
    event_df = event_description(event_df)
    event_df = Locations(event_df)
    event_df.drop(['eventId', 'positions', 'id', "event_tags", "tags", "eventId", "eventName"], inplace = True, axis = 1)
    event_df = event_df[event_df.subEventName != ""]
    return event_df
    
    
    

In [76]:
# Transform event_dfs

Events_France = event_df_transform(Events_France)
Events_Spain = event_df_transform(Events_Spain)
Events_Germany = event_df_transform(Events_Germany)
Events_Italy = event_df_transform(Events_Italy)
Events_England = event_df_transform(Events_England)



In [90]:
# Chris Jester-Young https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-values
def isNaN(num):
    return num != num

In [91]:
# this function fills in NaN values with correct teamId from the Matches df
def missing_teams(x):
    if np.isnan(x['team2']):
        row = Matches[Matches['matchId'] == x["matchId"]]
        if row['homeTeamId'].astype(int) == x['team1']:
            return int(row['awayTeamId'])
        else:
            return int(row['homeTeamId'])
    else:
        return x['team2']

In [ ]:
### remove events where playerId equals zero
## There are two instances where goals are scored so we will handle these first and assign to correct players as these are important events
# we can attempt to handle these into the real player sequences if we have time at the end of the project


In [380]:
eT =  "Corner" # convert to shorter name variable
# Get count for a player
df1 = pd.DataFrame(Events_England[Events_England["subEventName"]== eT ][['matchId', "teamId", 'playerId', "subEventName"]]).groupby(['matchId', 'playerId']).size().to_frame(eT).reset_index()

# get counts
pd.DataFrame(Events_England[Events_England["x"]










,matchId,playerId,Corner
0,2499719,3319,1
1,2499719,8013,1
2,2499719,26150,3
3,2499719,49876,7
4,2499720,8317,3
...,...,...,...
1450,2500098,8239,1
1451,2500098,8582,2
1452,2500098,9194,1
1453,2500098,70965,4


In [137]:
Teams[Teams['name']=="Manchester United"]

,city,name,wyId,officialName,area,type
79,Manchester,Manchester United,1611,Manchester United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club


In [350]:
Events_Italy[(Events_Italy['subEventDescription'].str.match('goal ')) & (Events_Italy['playerId']==0)]


In [427]:
England_Matches = Matches[Matches['Country']=='England']

In [464]:
trial = Events_England.groupby(['matchId','playerId', 'teamId']).agg({'xStart': ['mean'], 'yStart': ['mean']}).reset_index()
trial.columns = ["matchId", "playerId", 'teamId', "xStart", "yStart"]
trial['playerRef'] = None
trial = trial.merge(England_Matches, how='inner', on ='matchId')

trial['playerRef'] = trial.loc[:,'homePlayer1':'awayBenchPlayer7'].isin(trial['playerId']).idxmax(1)
trial = trial.iloc[:, :6]

trial.pivot(index='matchId', columns = [str(num)+"X" for num in range(1,12)], values=['xStart', 'yStart']).add_prefix('P').reset_index()

# Events_England = Events_England[Events_England['playerId']! = 0]



,matchId,playerId,teamId,xStart,yStart,playerRef
0,2499719,3319,1609,63.549020,43.901961,homePlayer9
1,2499719,3560,1609,34.153846,49.487179,homePlayer7
2,2499719,7868,1609,54.747573,38.058252,homePlayer6
3,2499719,7870,1609,61.450000,49.150000,homeBenchPlayer7
4,2499719,7879,1609,57.666667,85.000000,homeBenchPlayer6
...,...,...,...,...,...,...
10412,2500098,86325,1623,33.000000,37.666667,homePlayer1
10413,2500098,145692,1623,67.388060,58.134328,homePlayer1
10414,2500098,265318,1633,51.709091,16.309091,homeBenchPlayer7
10415,2500098,353236,1623,60.750000,52.283333,homePlayer1


In [ ]:
### TESTING for the below match aggs function

eT =  "Corner" # convert to shorter name variable
df1 = pd.DataFrame(Events_England[Events_England["subEventName"]== eT ][['matchId', "teamId", "subEventName"]])
df1 = df1.groupby(['matchId','teamId']).size().to_frame(eT).reset_index()
g = df1.groupby('matchId').agg(['unique']).reset_index()
f = pd.DataFrame(None, columns = ['matchId', 'teamId', eT])
f['matchId'], f['teamId'], f[eT] = g.iloc[:, 0], g.iloc[:,1], g.iloc[:,2]
h = pd.concat([f, pd.DataFrame(f['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
result = pd.merge(h, Matches[['matchId', "homeTeamId", "awayTeamId"]], on='matchId', how='inner')
result = pd.concat([result, pd.DataFrame(f[eT].to_list(), columns=[eT+"1",eT+"2"])], axis=1, sort = False)
result = result.fillna(0) # handle na values where teams had no corners in a match

# fill in zeros to teams where there were no corners for a team
result["Corner"] = result['Corner'].apply(lambda row: list(row)+[0] if len(row)==1 else row)

# Correctly apply allocate home and away aggregate statistics to correct teams
result['home'+eT] = (
    np.select(
        condlist=[result['team1'] == result['homeTeamId'], result['team2'] == result['homeTeamId']], 
        choicelist=[result[eT+"1"], result[eT+"2"]]))

result['away'+eT] = (
    np.select(
        condlist=[result['team1'] == result['awayTeamId'], result['team2'] == result['awayTeamId']], 
        choicelist=[result[eT+"1"], result[eT+"2"]]))


# # drop useless columns that have been replaced
result.drop(['teamId', eT, "team1", "team2", eT+"1", eT+"2"], inplace = True, axis = 1 )


In [92]:
# Function to transform match events into meaningul player performance profiles for machine learning
def match_agg(eT, event_df):

    df1 = pd.DataFrame(event_df[event_df["subEventName"]== eT ][['matchId', "teamId", "subEventName"]])
    df1 = df1.groupby(['matchId','teamId']).size().to_frame(eT).reset_index()
    df2 = df1.groupby('matchId').agg(['unique']).reset_index()
    df3 = pd.DataFrame(None, columns = ['matchId', 'teamId', eT])
    df3['matchId'], df3['teamId'], df3[eT] = df2.iloc[:, 0], df2.iloc[:,1], df2.iloc[:,2]
    df4 = pd.concat([df3, pd.DataFrame(df3['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
    result = pd.merge(df4, Matches[['matchId', "homeTeamId", "awayTeamId"]], on='matchId', how='inner')
    result = pd.concat([result, pd.DataFrame(df3[eT].to_list(), columns=[eT+"1",eT+"2"])], axis=1, sort = False)
    result = result.fillna(0) # handle na values where teams had no corners in a match

    # fill in zeros to teams where there were no corners for a team
    result[eT] = result[eT].apply(lambda row: list(row)+[0] if len(row)==1 else row)

    # Correctly apply allocate home and away aggregate statistics to correct teams
    result['home'+eT] = (
        np.select(
            condlist=[result['team1'] == result['homeTeamId'], result['team2'] == result['homeTeamId']], 
            choicelist=[result[eT+"1"], result[eT+"2"]]))

    result['away'+eT] = (
        np.select(
            condlist=[result['team1'] == result['awayTeamId'], result['team2'] == result['awayTeamId']], 
            choicelist=[result[eT+"1"], result[eT+"2"]]))
    
    # Create a column to recognize when a goal is scored
    conditions = [
        (((result['subEventDescription'].str.match('goal')) & (result['Result'] == "Success"))),
        (((result['subEventDescription'].str.match('goal')) & (result['Result'] == "Failure") & (result['subEventName'] == "Shot"))),
        (((result['subEventDescription'].str.match('goal')) & (result['Result'] == "Failure") & (result['subEventName'] != "Shot")))
        ]

    # create a list of the values we want to assign for each condition
    values = [1, 1, -1]

    # create a new column and use np.select to assign values to it using our lists as arguments
    result['Goal_Value'] = np.select(conditions, values, default = 0)
    
    result = result[result['playerId']!= 0]

    # # drop useless columns that have been replaced
    result.drop(['teamId', eT, "team1", "team2", eT+"1", eT+"2"], inplace = True, axis = 1 )
    
    return result

In [ ]:
# DEPRECATED 

# def sortHomeAway(y, eT):
    
#     # homeTeamId, awayTeamId, homeFeature, awayFeature
#     l1, l2, l3, l4 = [], [], [], []
#     for index, x in y.iterrows():
#         row = Matches[Matches['matchId'] == x["matchId"]]
#         if int(row['homeTeamId']) == x['team1']:
#             l1.append(int(x['team1'])), l2.append(int(x['team2'])), l3.append(int(x["home" + eT])), l4.append(int(x["away" + eT]))
#         else:
#             l2.append(int(x['team1'])), l1.append(int(x['team2'])), l4.append(int(x["home" + eT])), l3.append(int(x["away" + eT]))
#     return l1,l2,l3,l4



In [102]:
# generate comprehensive dataframe with summary stat for a particular team-match attribute
a = match_agg("Corner", Events_England) 
b = match_agg("Corner", Events_France)
c = match_agg("Corner", Events_Italy)
d = match_agg("Corner", Events_Spain)
e = match_agg("Corner", Events_Germany) 
#f = match_agg("Corner", Events_World_Cup) 
#g = match_agg("Corner", Events_EuroChamps) 

match_aggs = pd.concat([a,b,c,d,e], ignore_index=True)


In [112]:
Events_England

,subEventName,playerId,matchId,teamId,matchPeriod,eventSec,subEventId,Result,subEventDescription,xStart,xEnd,yStart,yEnd
0,Simple pass,25413,2499719,1609,1H,2.758649,85,Success,generic play,49,31,49,78
1,High pass,370224,2499719,1609,1H,4.946850,83,Success,generic play,31,51,78,75
2,Head pass,3319,2499719,1609,1H,6.542188,82,Success,generic play,51,35,75,71
3,Head pass,120339,2499719,1609,1H,8.143395,82,Success,generic play,35,41,71,95
4,Simple pass,167145,2499719,1609,1H,10.302366,85,Success,generic play,41,72,95,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
643143,Cross,14703,2500098,1633,2H,2789.232666,80,Failure,right_foot blocked,99,0,72,0
643146,Corner,70965,2500098,1633,2H,2829.821084,30,Success,key_pass high,100,88,100,47
643147,Air duel,7919,2500098,1623,2H,2831.211419,10,Failure,lost,12,14,53,50
643148,Air duel,8005,2500098,1633,2H,2832.434399,10,Success,won,88,86,47,50


In [108]:
adv_stats

['aerialDuelsWon',
 'defensiveDuelsWon',
 'directFreeKicksOnTarget',
 'duelsWon',
 'fieldAerialDuelsWon',
 'gkAerialDuelsWon',
 'gkSaves',
 'gkSuccessfulExits',
 'goalConversion',
 'headShotsOnTarget',
 'newDefensiveDuelsWon',
 'newDuelsWon',
 'newOffensiveDuelsWon',
 'newSuccessfulDribbles',
 'offensiveDuelsWon',
 'penaltiesConversion',
 'shotsOnTarget',
 'successfulBackPasses',
 'successfulCrosses',
 'successfulDribbles',
 'successfulForwardPasses',
 'successfulKeyPasses',
 'successfulLateralPasses',
 'successfulLinkupPlays',
 'successfulLongPasses',
 'successfulPasses',
 'successfulPassesToFinalThird',
 'successfulShotAssists',
 'successfulSmartPasses',
 'successfulThroughPasses',
 'successfulTouchInBox',
 'successfulVerticalPasses',
 'win',
 'yellowCardsPerFoul',
 'accelerations',
 'aerialDuels',
 'aerialDuelsWon',
 'assists',
 'attackingActions',
 'backPasses',
 'cleanSheets',
 'concededGoals',
 'corners',
 'crosses',
 'dangerousOpponentHalfRecoveries',
 'dangerousOwnHalfLosses',


In [107]:
# list of advanced summary stats
adv_stats = ["aerialDuelsWon",  "defensiveDuelsWon",  "directFreeKicksOnTarget",  "duelsWon",  "fieldAerialDuelsWon",  "gkAerialDuelsWon",  "gkSaves",  "gkSuccessfulExits",  "goalConversion",  "headShotsOnTarget",  "newDefensiveDuelsWon",  "newDuelsWon",  "newOffensiveDuelsWon",  "newSuccessfulDribbles",  "offensiveDuelsWon",  "penaltiesConversion",  "shotsOnTarget",  "successfulBackPasses",  "successfulCrosses",  "successfulDribbles",  "successfulForwardPasses",  "successfulKeyPasses",  "successfulLateralPasses",  "successfulLinkupPlays",  "successfulLongPasses",  "successfulPasses",  "successfulPassesToFinalThird",  "successfulShotAssists",  "successfulSmartPasses",  "successfulThroughPasses",  "successfulTouchInBox",  "successfulVerticalPasses",  "win",  "yellowCardsPerFoul",  "accelerations",  "aerialDuels",  "aerialDuelsWon",  "assists",  "attackingActions",  "backPasses",  "cleanSheets",  "concededGoals",  "corners",  "crosses",  "dangerousOpponentHalfRecoveries",  "dangerousOwnHalfLosses",  "defensiveActions",  "defensiveDuels",  "defensiveDuelsWon",  "directFreeKicks",  "directFreeKicksOnTarget",  "directRedCards",  "dribbles",  "duels",  "duelsWon",  "fieldAerialDuels",  "fieldAerialDuelsWon",  "forwardPasses",  "fouls",  "freeKicks",  "freeKicksOnTarget",  "gkAerialDuels",  "gkAerialDuelsWon",  "gkExits",  "gkGoalKicks",  "gkGoalKicksSuccess",  "gkSaves",  "gkSuccessfulExits",  "goals",  "headShots",  "interceptions",  "keyPasses",  "lateralPasses",  "linkupPlays",  "longGoalKicks",  "longPasses",  "looseBallDuels",  "looseBallDuelsWon",  "losses",  "matches",  "matchesTagged",  "missedBalls",  "newDefensiveDuelsWon",  "newDuelsWon",  "newOffensiveDuelsWon",  "newSuccessfulDribbles",  "offensiveDuels",  "offensiveDuelsWon",  "offsides",  "opponentHalfRecoveries",  "opponentOffsides",  "ownHalfLosses",  "passes",  "passesToFinalThird",  "penalties",  "pressingDuels",  "pressingDuelsWon",  "progressiveRun",  "receivedPass",  "recoveries",  "redCards",  "shortGoalKicks",  "shotAssists",  "shotOnTargetAssists",  "shots",  "shotsAgainst",  "smartPasses",  "successfulAttackingActions",  "successfulBackPasses",  "successfulCrosses",  "successfulDefensiveActions",  "successfulDribbles",  "successfulForwardPasses",  "successfulKeyPasses",  "successfulLateralPasses",  "successfulLinkupPlays",  "successfulLongPasses",  "successfulPasses",  "successfulPassesToFinalThird",  "successfulPenalties",  "successfulSmartPasses",  "successfulThroughPasses",  "successfulVerticalPasses",  "throughPasses",  "touchInBox",  "verticalPasses",  "xgShot",  "xgShotAgainst",  "yellowCards",  "accelerations",  "aerialDuels",  "assists",  "attackingActions",  "backPasses",  "ballLosses",  "ballRecoveries",  "clearances",  "corners",  "counterpressingRecoveries",  "crosses",  "dangerousOpponentHalfRecoveries",  "dangerousOwnHalfLosses",  "defensiveActions",  "defensiveDuels",  "defensiveDuelsWon",  "directFreeKicks",  "directFreeKicksOnTarget",  "directRedCards",  "dribbleDistanceFromOpponentGoal",  "dribbles",  "dribblesAgainst",  "dribblesAgainstWon",  "duels",  "duelsWon",  "fieldAerialDuels",  "fieldAerialDuelsWon",  "forwardPasses",  "fouls",  "foulsSuffered",  "freeKicks",  "freeKicksOnTarget",  "gkAerialDuels",  "gkAerialDuelsWon",  "gkConcededGoals",  "gkExits",  "gkSaves",  "gkShotsAgainst",  "gkSuccessfulExits",  "goalKicks",  "goalKicksLong",  "goalKicksShort",  "goals",  "headShots",  "interceptions",  "keyPasses",  "lateralPasses",  "linkupPlays",  "longPassLength",  "longPasses",  "looseBallDuels",  "looseBallDuelsWon",  "missedBalls",  "newDefensiveDuelsWon",  "newDuelsWon",  "newOffensiveDuelsWon",  "newSuccessfulDribbles",  "offensiveDuels",  "offensiveDuelsWon",  "offsides",  "opponentHalfRecoveries",  "ownHalfLosses",  "passLength",  "passes",  "passesToFinalThird",  "penalties",  "progressivePasses",  "progressiveRun",  "receivedPass",  "redCards",  "secondAssists",  "shotAssists",  "shotOnTargetAssists",  "shots",  "shotsBlocked",  "shotsOnTarget",  "slidingTackles",  "smartPasses",  "successfulAttackingActions",  "successfulBackPasses",  "successfulCrosses",  "successfulDefensiveAction",  "successfulDribbles",  "successfulForwardPasses",  "successfulGoalKicks",  "successfulKeyPasses",  "successfulLateralPasses",  "successfulLinkupPlays",  "successfulLongPasses",  "successfulPasses",  "successfulPassesToFinalThird",  "successfulPenalties",  "successfulProgressivePasses",  "successfulSlidingTackles",  "successfulSmartPasses",  "successfulThroughPasses",  "successfulVerticalPasses",  "thirdAssists",  "throughPasses",  "touchInBox",  "verticalPasses",  "xgAssist",  "xgSave",  "xgShot",  "yellowCards",  "avgPossessionDuration",  "minutesOfPossession1-15",  "minutesOfPossession106-120",  "minutesOfPossession16-30",  "minutesOfPossession31-45",  "minutesOfPossession46-60",  "minutesOfPossession61-75",  "minutesOfPossession76-90",  "minutesOfPossession91-105",  "possession1-15",  "possession106-120",  "possession16-30",  "possession31-45",  "possession46-60",  "possession61-75",  "possession76-90",  "possession91-105",  "possessionNumber",  "possessionPercent",  "purePossessionTime",  "reachingOpponentBox",  "reachingOpponentHalf",  "avgPossessionDuration",  "minutesOfPossession1-15",  "minutesOfPossession106-120",  "minutesOfPossession16-30",  "minutesOfPossession31-45",  "minutesOfPossession46-60",  "minutesOfPossession61-75",  "minutesOfPossession76-90",  "minutesOfPossession91-105",  "possession1-15",  "possession106-120",  "possession16-30",  "possession31-45",  "possession46-60",  "possession61-75",  "possession76-90",  "possession91-105",  "possessionNumber",  "possessionPercent",  "purePossessionTime",  "reachingOpponentBox",  "reachingOpponentHalf",  "assists",  "avgPassLength",  "avgPassToFinalThirdLength",  "backPasses",  "backPassesSuccessful",  "crossesBlocked",  "crossesFromLeftFlank",  "crossesFromLeftFlankSuccessful",  "crossesFromRightFlank",  "crossesFromRightFlankSuccessful",  "crossesHigh",  "crossesLow",  "crossesSuccessful",  "crossesTotal",  "deepCompletedPasses",  "deepCompletedPassesSuccessful",  "forwardPasses",  "forwardPassesSuccessful",  "keyPasses",  "keyPassesSuccessful",  "lateralPasses",  "lateralPassesSuccessful",  "longPasses",  "longPassesSuccessful",  "matchTempo",  "passToFinalThirds",  "passToFinalThirdsSuccessful",  "passToPenaltyAreas",  "passToPenaltyAreasSuccessful",  "passes",  "passesSuccessful",  "progressivePasses",  "progressivePassesSuccessful",  "shortMediumPasses",  "shortMediumPassesSuccessful",  "shotAssists",  "smartPasses",  "smartPassesSuccessful",  "throughPasses",  "throughPassesSuccessful",  "verticalPasses",  "verticalPassesSuccessful",  "corners",  "cornersWithShot",  "counterAttacks",  "freeKicks",  "freeKicksWithShot",  "positionalAttack",  "positionalWithShots",  "total",  "withShots",  "clearances",  "interceptions",  "ppda",  "tackles",  "aerialDuels",  "aerialDuelsSuccessful",  "challengeIntensity",  "defensiveDuels",  "defensiveDuelsSuccessful",  "dribbles",  "dribblesSuccessful",  "duels",  "duelsSuccessful",  "groundDuels",  "groundDuelsSuccessful",  "looseBallDuels",  "looseBallDuelsSuccessful",  "offensiveDuels",  "offensiveDuelsSuccessful"]



In [ ]:
Events_England.subEventName.unique()



In [ ]:
test = pd.merge(Matches, match_aggs, how = 'left', left_on = ['matchId', 'homeTeamId', 'awayTeamId'], right_on = ['matchId', 'homeTeamId', 'awayTeamId'])

In [ ]:
Events_Spain[Events_Spain["playerId"]==8278]

In [ ]:
Players[Players["lastName"]=="Bale"]

In [ ]:
Events_England['subEventName'].unique()

In [80]:
Teams[Teams['name']=="Manchester United"]

,city,name,wyId,officialName,area,type
79,Manchester,Manchester United,1611,Manchester United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club


In [ ]:
Events_England.groupby(['playerId', 'matchId']).agg({'Age': ['mean', 'min', 'max']})



In [291]:
Events_England.subEventDescription.unique()

array(['generic play', 'lost', 'won', 'interception', 'through',
       'take_on_right lost', 'take_on_left won', 'free_space_right won',
       'free_space_left lost', 'free_space_left sliding_tackle lost',
       'free_space_left won', 'free_space_right lost',
       'assist right_foot high',
       'goal right_foot opportunity position_goal_low_left',
       'goal position_goal_low_left',
       'left_foot opportunity position_out_mid_left',
       'counter_attack interception',
       'counter_attack free_space_left won',
       'counter_attack left_foot blocked', 'dangerous_ball_lost lost',
       'left_foot', 'neutral', 'right_foot high', 'assist',
       'goal head/body opportunity position_goal_high_center',
       'goal position_goal_high_center',
       'left_foot opportunity position_out_high_left',
       'take_on_right won', 'take_on_left lost', 'left_foot high', 'high',
       'take_on_right neutral', 'take_on_left neutral',
       'free_space_right sliding_tackle lost',


### Attempt to design a "danger index" for each action and then aggregate for players and teams
##### Should also contemplate design for chemistry index, and Goalproof index

In [ ]:
# (-15)**2*-0.05

In [ ]:
# (trial['xEnd']- trial['xStart'])/100

In [ ]:
# trial["Danger_index"] = (trial['yEnd']**2) * (trial['xEnd']- trial['xStart'])/100

In [ ]:
# trial[0:3]

In [ ]:
trial[(trial['subEventName'] == "Shot") & (trial['Result']=="Failure")]

In [ ]:
trial[0:2]['EndLoc'][0]['y'] == 100

In [ ]:
Goal = []
for index, row in trial.iterrows():
    if 'goal' in row['subEventDescription'] and row['EndLoc']['x'] == 100:
        Goal.append(1)
    else:
        Goal.append(0)
trial["ScoresGoal"] = Goal

In [ ]:
trial

In [ ]:
# EDIT MATCH TIME COLUMN

In [ ]:
try:
    os.chdir('/Users/jordanmakins/Desktop/Data/events')
except:
    pass
Events_England = pd.read_json('events_England.json')

In [ ]:
trial2 = Events_England[0:4000]

In [ ]:
# Solution to convert second half time into 
secs_to_add = list(trial2[trial2['matchPeriod']=="1H"].groupby('matchId').tail(1)['eventSec'])
match_ids = list(trial2["matchId"].unique())

for idx in range(0, len(match_ids)):
    trial2['eventSec'] = list(np.where(
       (trial2['matchId'] == match_ids[idx]) & (trial2['matchPeriod'] == "2H") , trial2['eventSec'] + secs_to_add[idx], trial2['eventSec']
       ))
    

In [ ]:
# Add a column for euc

,venue,matchId,competitionId,Country,homeScore,awayScore,homeScoreHT,awayScoreHT,homeScoreET,awayScoreET,...,awayBenchPlayer3,awayBenchPlayer4,awayBenchPlayer5,awayBenchPlayer6,awayBenchPlayer7,homeManager,homeTeamId,awayManager,awayTeamId,Result
0,Turf Moor,2500089,364,England,1,2,1,0,0,0,...,20921,515791,7989.0,8296.0,11669.0,8880,1646,8934,1659,-1
1,Selhurst Park,2500090,364,England,2,0,0,0,0,0,...,8362,8096,7907.0,261.0,8686.0,8357,1628,0,1627,1
2,The John Smith's Stadium,2500091,364,England,0,1,0,1,0,0,...,346159,171166,7945.0,3560.0,279717.0,18572,1673,7845,1609,-1
3,Anfield,2500092,364,England,4,0,2,0,0,0,...,3278,4255,15526.0,8416.0,13451.0,14791,1612,8093,1651,1
4,Old Trafford,2500093,364,England,1,0,1,0,0,0,...,14688,234408,8889.0,7888.0,8903.0,3295,1611,93112,1644,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,Parc des Princes,1694408,102,European Champs,0,1,0,1,0,0,...,212604,69417,284470.0,69404.0,284315.0,32637,4687,69952,9598,-1
1937,Stade V\u00e9lodrome,1694397,102,European Champs,1,1,0,0,0,0,...,101611,101663,102083.0,101699.0,101704.0,8357,2413,101575,14358,0
1938,Stade Matmut-Atlantique,1694396,102,European Champs,2,1,1,0,0,0,...,104937,101555,138408.0,32803.0,101813.0,136439,10682,268998,14496,1
1939,Stade Bollaert-Delelis,1694391,102,European Champs,0,1,0,1,0,0,...,14914,360992,49919.0,134911.0,236528.0,135480,8731,210701,6697,-1


### First Attempt with Hugo Mathien

In [ ]:
# cursor = sql_conn.cursor()
# table_names = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

In [ ]:
# sql_conn2 = sqlite3.connect('hugomathienData.sqlite') # soccer database containing over 100,000 match results which is updated monthly
# cursor2 = sql_conn2.cursor()
# table_names = cursor2.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor2.fetchall())

In [ ]:
# Country = pd.read_sql_query("SELECT * FROM Country", sql_conn2)
# League = pd.read_sql_query("SELECT * FROM League", sql_conn2)
# Match_hugo = pd.read_sql_query("SELECT * FROM Match", sql_conn2)
# Player = pd.read_sql_query("SELECT * FROM Player", sql_conn2)
# Player_Attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", sql_conn2)
# Team = pd.read_sql_query("SELECT * FROM Team", sql_conn2)
# Team_Attributes = pd.read_sql_query("SELECT * FROM Team_Attributes", sql_conn2)

### Data Processing

In [ ]:
#Match_TeamStats = pd.read_sql_query("SELECT * FROM football_data", sql_conn)

In [ ]:
# # parsing betting odds variables from analysis
# drop_columns = ["B365H","B365D","B365A","BSH","BSD","BSA","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA",
# "PSH", "PH","PSD", "PD","PSA", "PA","SOH","SOD","SOA","SBH","SBD","SBA","SJH","SJD","SJA","SYH","SYD","SYA","VCH",
# "VCD","VCA","WHH","WHD","WHA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","MaxH","MaxD","MaxA",
# "AvgH","AvgD","AvgA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","GB>2.5","GB<2.5","B365>2.5","B365<2.5",
# "P>2.5","P<2.5","Max>2.5","Max<2.5","Avg>2.5","Avg<2.5","BbAH","BbAHh","AHh","BbMxAHH","BbAvAHH","BbMxAHA",
# "BbAvAHA","GBAHH","GBAHA","GBAH","LBAHH","LBAHA","LBAH","B365AHH","B365AHA","B365AH","PAHH", "AvgC<2.5",
# "PAHA","MaxAHH","MaxAHA","AvgAHH","AvgAHA","WHCH","WHCD","WHCA", "VCCH", "VCCA", "PSCH", "PSCD", "PCAHH", "PCAHA", "PC>2.5", "PC<2.5", "MaxCH", "MaxCAHH", "MaxCD",
# "MaxCAHA", "MaxCA", "MaxC>2.5", "MaxC<2.5", "IWCH", "IWCD", "IWCA", "AvgC>2.5","AvgCA","AvgCAHA","AvgCAHH",
# "AvgCD","AvgCH","B365C<2.5","B365C>2.5","B365CA","B365CAHA","B365CAHH","B365CD","B365CH","BWCA", "BWCD", "BWCH", "VCCD", "PSCA"] 

In [ ]:
# Match_TeamStats.drop(drop_columns, inplace=True, axis =1) # removing betting odds from dataframe

In [ ]:
def df_analysis(df, df_name):
    header = "Dimension of " + str(df_name) + " Dataframe is: {}"
    print(header.format(df.shape))
    print(100*"*")
    print(df.info())
    print(100*"*")
    print(df.select_dtypes(exclude=['float64','int64']).describe())
    print(100*"*")
    print(df.describe())
    print(100*"*")
    print("Number of Null Values")
    print(df.isnull().sum(axis=0))

In [ ]:
# df_analysis(Match_TeamStats, "Match_TeamStats")

In [ ]:
# match[match["HomeTeam"].str.contains("Man United")]

### Outlier Detection and Understanding